# Основы NumPy: массивы и векторные вычисления. Функции активации

Данная лабораторная работа посвящена изучению основного пакета для численных расчетов в Python - [NumPy](https://numpy.org/). Хоть пакет NumPy почти не содержит в себе средств для моделирования и научных расчетов, понимание массивов NumPy и ориентированных на эти массивы вычислений поможет гораздо эффективнее использовать инструменты типа [pandas](https://pandas.pydata.org/). Основными аспектами NumPy для эффективной работы с большими массивами данных являются:
- хранение данных в непрерывном блоке памяти независимо от других встроенных объектов Python, что позволяет работать с этим блоков без проверки типов данных и других накладных расходов. Массивы NumPy занимают гораздо меньше памяти, чем встроенные в Python последовательности;
- сложные операции применяются к массивам целиком, так что циклы for не нужны.

**Цели лабораторной:**
- Научиться использовать iPython Notebooks
- Познакомиься с numpy-функциями и векторными вычислениями
- Понять механизм *укладывания* (broadcasting) массивов
- Научиться векторизировать вычисления

Давайте приступем!

## О работе с iPython Notebooks ##

iPython Notebooks - это интерактивный веб-блокнот (в рускоязычной среде обычно переводят как блокнот, тетрадка, ноутбук), представляющий продуктивную среду для интерактивных исследовательских вычислений. В 2014 году веб-блокнот IPython превратился в Jupyter-блокнот, который ныне поддерживает более 40 языково-независимых средств интерактивных вычислений.

В основном в каждом блокноте курса для создания заданий использовалось расширение nbgrader, автоматически создающее участки кода с заданием. Участки, требующие от Вас написания кода отмечены строчкой `### YOUR CODE HERE raise NotImplementedError()`. Напишите вместо этого ваш код и запустите его сочетанием клавиш "SHIFT"+"ENTER" или нажав "Run Cell" на панели инструментов. Вы не ограничены числом строчек для написания кода, однако старайтесь писать код максимально лаконично и с использованием только тех библиотек, что определены в начале всего блокнота. Например, при работе с массивами NumPy следует по возможности использовать встроенные векторизированные функции, а не применять циклы.

Часто после задания используются нередактируемые блоки проверки правильности выполнения, включающие в себя функции `assert`. Всегда выполняйте данные блоки!Не смотря на то, что их результаты не влияют на дельнейший код, они помогают в первую очередь Вам проверить свое решение, а также автоматически начисляют баллы за правильность в системе автоматизированной проверки.

Еще одним типом задания является развернутый ответ на вопрос, который следует писать в ячейке `YOUR ANSWER HERE`, где для написания текста следует пользоваться разметкой *[Markdown](https://www.markdownguide.org)*, а для формул - LaTeX: $\sum_{i=1}^n i^2$ (просто отделив LaTeX-выражение с помощью $-символов).

## Формальный нейрон ##

Искусственные нейронные сети появились в результате применения математического аппарата к исследованию функционирования нервной системы. Полученные при этом результаты успешно применяются при решении проблем распознавания образов, моделирования, прогнозирования, оптимизации и управления.
Основной структурной и функциональной частью нейронной сети является **формальный нейрон (formal neuron)**, представленный на
рисунке, где $x_0,x_1,...,x_n$ – компоненты вектора входных сигналов,
$w_0,w_1,...,w_n$ – значения весов входных сигналов нейрона, а $y$ – выходной сигнал нейрона:

![Формальный нейрон](https://drive.google.com/uc?id=1Vzl3XDa3ApBJVkvIau9XMBEXHv3UwRyk)

Формальный нейрон состоит из элементов 3 типов: умножителей (синапсов), сумматора и преобразователя. Синапс характеризует силу (вес) связи между двумя нейронами. Сумматор выполняет сложение входных сигналов, предварительно помноженных на соответствующие веса. Преобразователь реализует функцию одного аргумента – выхода сумматора. Эта функция называется **функцией активации** или **передаточной функцией нейрона**. Исходя из данного описания, математическая модель нейрона может быть представлена следующим образом:
$$y=f(S),\\
S=\sum_{i=1}^n w_i x_i + b $$


Наиболее часто используемые функции активации представлены в таблице 1:

|Название|Формула|
|:--------------|:----------------------------------------------------------|
|Пороговая      |$$f(S)=\begin{cases} 0,S<\theta\\1,S\ge\theta \end{cases}$$|
|Линейная       |$$f(S)=aS$$                                                |
|Лог-сигмоидная |$$f(S) = \frac{1}{1+e^{-aS}}$$                             |
|Гиперболический<br>тангенс|$$f(S) = \frac{e^{aS}-e^{-aS}}{e^{aS}+e^{-aS}}$$|
|ReLU           |$$f(S)=\max(S,0)$$                                         |

## 1 - Базовые функции NumPy. Реализация функций активации ##

### 1.1 - Объект ndarray ###

Одна из ключевых особенностей NumPy - объект ndarray для представления N-мерного массива. Это быстрый и гибкий контейнер для хранения больших наборов данных в Python. Массивы позволяют выполнять математические операции над целыми блоками данных, применяя такой же синтаксис, как для соотвествующих операций над скалярами.

В машинном обучении часто используется слово **тензор** - это контейнер для чисел. В терменологии тензоров *измерения* часто называют *осями*. Количество осей тензора называют его *рангом*.
Рассмотрим виды тензоров:
- тензор, содержащий единственное число, называется *скаляром*: `np.array(12)`
- тензор первого ранга - одномерный массив чисел, называют *вектором*: `np.array([12,5,6,20])`
- тензор второго ранга - массив векторов - это *матрица*: `np.array([[5,9],[6,3]])`
- тензор третьего и высшего ранга: `np.array([[[1,2],[3,4]],[[5,6],[7,8]]])`

Тензор определяется тремя ключевыми атрибутами:
- количество осей(ранг): `ndim`
- форма - кортеж целых чисел, описывающих количество измерений на каждой оси тензора: `shape`
- тип данных, содержащихся в тензоре: `dtype`

In [ ]:
import numpy as np
x = np.array([[[1,2],[3,4],[5,6]],[[7,8],[9,10],[11,12]]])
print(x)
print("Ранг: ", x.ndim)
print("Форма: ", x.shape)
print("Тип данных: ", x.dtype)

### 1.2 - Сигмоидальная функция активации ###

В данном задании Вам пригодится функция [np.exp()](https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.exp.html), вычисляющая экспоненту числа, однако до этого Вы использовали [math.exp()](https://docs.python.org/3/library/math.html). Для быстрого вызова документации по функции вы можете в отдельной ячейке кода вызвать команду без параметров и с ? знаком, например: `np.exp?`.

**Задание**: Реализуйте две сигмоидальные функции с помощью библиотек math и numpy, которые на вход принимают тензор x любого ранга и формы, и сравните их результаты.
$$ \text{Для } x \in \mathbb{R}^n \text{,     } sigmoid(x) = sigmoid\begin{pmatrix}
    x_1  \\
    x_2  \\
    ...  \\
    x_n  \\
\end{pmatrix} = \begin{pmatrix}
    \frac{1}{1+e^{-x_1}}  \\
    \frac{1}{1+e^{-x_2}}  \\
    ...  \\
    \frac{1}{1+e^{-x_n}}  \\
\end{pmatrix}\tag{1} $$

In [ ]:
# Вычисление сигмоиды скалярного значения
import math

def basic_sigmoid(x):
    # YOUR CODE HERE
    raise NotImplementedError()
    return s

In [ ]:
assert round(basic_sigmoid(3), 2) == 0.95

Вычисление функции math.exp для вектора приведет к ошибке.

In [ ]:
### Попробуйте вычислить результат для вектора ###
x = [1, 2, 3]
basic_sigmoid(x)

Библиотека numpy позволит провести вычисление функции для векторов. Реализуйте функцию sigmoid с использованием библиотеки numpy и функции numpy.exp.

In [ ]:
# Реализуйте функцию sigmoid
import numpy as np

def sigmoid(x):
    # YOUR CODE HERE
    raise NotImplementedError()

    return s

In [ ]:
x = np.array([1, 2, 3])
res = sigmoid(x)
assert round(res[0], 2) == 0.73
assert round(res[1], 2) == 0.88
assert round(res[2], 2) == 0.95

### 1.3 - Производная сигмоидальной функции

Для выполнения дальнейших лабораторных работ Вам понадобится функция, вычисляющая производную сигмоидальной функции, в частности для оптимизации функции стоимости методом обратного распространения ошибки.

**Задание**: Реализуйте функцию sigmoid_grad(), вычисляющую первую производную сигмоидальной функции по входному зачению x:
$$sigmoid\_derivative(x) = \sigma'(x) = \sigma(x) (1 - \sigma(x))\tag{2}$$

In [ ]:
def sigmoid_derivative(x):
    # YOUR CODE HERE
    raise NotImplementedError()

    return ds

In [ ]:
x = np.array([1, 2, 3])
res = sigmoid_derivative(x)
print ("sigmoid_derivative(x) = " + str(res))
assert round(res[0], 2) == 0.20
assert round(res[1], 2) == 0.10
assert round(res[2], 3) == 0.045

### 1.4 - Визуализация графиков функций активации ###

Понимание области значений и вида графика функции активации обязательно для дальнейшего изучения грамотного построения нейронных сетей. Приведем пример получения графика сигмоидальной функции активации.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')
x = np.linspace(-10, 10, 100)
plt.plot(x, sigmoid(x))
plt.show()

**Задание**: Реализуте остальные 4 функции активации, визуализируйте их графики на одном plot с помощью [subplots](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html#matplotlib.pyplot.subplots), а также скопируйте таблицу 1 в поле `YOUR ANSWER HERE` и дополните колонкой "Область значений", заполните ее.

YOUR ANSWER HERE

### 1.5 - Изменение формы массива ###

Во многих случаях изменить форму массива можно без копирования данных. Для этого следует передать кортеж с описанием новой формы методу экземпляра массива функции [np.reshape()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html).

Изменение формы с преобразованием в двумерный массив:
- порядок по строкам (как в С): старшие измерения обходятся *раньше* (т.е сначала обойти ось 1, а потом переходить к оси 0);
- порядок по столбцам (как в Fortran): старшие измерения обходятся *позже* (т.е. сначала обойти ось 0, а потом переходить к оси 1).

<img src="https://drive.google.com/uc?id=19XcTU1_iocjzSeNVklKIxsw_DzGjGvAe" width=60%>

По умолчанию массивы NumPy размещаются в памяти *по строкам*.

Например, изображение формата RGB в компьютере представляет собой трехмерный массив формы $(ширина, высота, цвет = 3)$. Однако, для дальнейшей работы с изображениями Вам необходимо преобразовать их к форме $(ширина*высота*3, 1)$. Другими словами, Вы должны "развернуть", трехмерный массив в вектор значений.

<img src="https://drive.google.com/uc?id=1UvEFu6zadF1WcWk_ny8_IyxKzYTANVBS" width=65%>

**Задание**: Реализуйте функцию `image_to_vector()`, которая принимает на вход тензор формы $(ширина, высота, 3)$ и возвращает вектор данных формы $(ширина*высота*3, 1)$. Другими словами, измените форму тензора $v$ с $(a, b, c)$ на $(a*b,c)$.

In [ ]:
def image_to_vector(image):
    # YOUR CODE HERE
    raise NotImplementedError()

    return v

In [ ]:
image = np.array([[[ 0.67826139,  0.29380381],
        [ 0.90714982,  0.52835647],
        [ 0.4215251 ,  0.45017551]],

       [[ 0.92814219,  0.96677647],
        [ 0.85304703,  0.52351845],
        [ 0.19981397,  0.27417313]],

       [[ 0.60659855,  0.00533165],
        [ 0.10820313,  0.49978937],
        [ 0.34144279,  0.94630077]]])
res = image_to_vector(image)
print ("image_to_vector(image) = " + str(res))

assert round(res[5,0], 2) == 0.45
assert round(res[8,0], 2) == 0.85
assert round(res[2,0], 2) == 0.91

**Задание**: какие две функции numpy используются для *линеаризации* - перехода от многомерного массива к одномерному массиву - и в чем их отличие?

YOUR ANSWER HERE

### 1.6 - Укладывание ####
Словом "укладывание" ([broadcasting](http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)) Описывается способ выполнения арифметических операций над массивами разной формы. Это очень мощный механизм, но даже опытные пользователи иногда испытывают затруднения с его пониманием. Простейший пример укладывания - комбинирование скалярного значения с массивом:

In [ ]:
#Укладывание скалярного значения 4 на все элементы массива в результате умножения
arr = np.arange(6)
print(arr)
print(arr * 4)

**Правило укладывания**: Два массива совместимы по укладыванию, если для обоих *последних измерений* (т.е. отсчитываемых с конца) длины осей совпадают или хотя бы одна длина равна 1. Тогда укладывание производится по отсутствующим измерениям или по измерениям длины 1.

Укладывание одномерного массива по оси 0:
<img src="https://drive.google.com/uc?id=1Monc6gotuHofKH0GpJM9D_-a_mJvs5qF" title="Укладывание одномерного массива по оси 0" width="75%">

In [ ]:
arr = np.arange(4).repeat(3)
arr = arr.reshape(4,3)
print('arr:\n',arr)
arr2 = np.arange(1,4)
print('arr2:\n',arr2)
print('arr + arr2:\n',arr + arr2)

Укладывание одномерного массива по оси 1:
<img src="https://drive.google.com/uc?id=1XqruihvMYYGgjcEXBKGLlid3RJKWCH8j" title="Укладывание одномерного массива по оси 1" width="75%">

In [ ]:
arr2 = np.arange(1,5)
arr2 = arr2.reshape(4,1)
print('arr:\n',arr)
print('arr2:\n',arr2)
print('arr + arr2:\n',arr + arr2)

В трехмерном случае укладываниепо любому из трех измерений сводится к изменению формы данных для обеспечения совместимости массивов. На рисунке наглядно показано, каковы должны быть формы для укладывания по любой оси трехмерного массива.
Укладывание двумерного массива в трехмерный массив:
<img src="https://drive.google.com/uc?id=1o72i2gdo9NOLmpfQUdxD0g6R6FGOeN2b" title="Укладывание двумерного массива в трехмерный массив" width="65%">

**Задание**: Реализуйте функцию softmax с помощью numpy и применяя укладывание массивов. Softmax — это логистическая функция для многомерного случая. Это означает, что функцию применяют не к отдельному значению, а к вектору. Например, её можно использовать в том случае, когда стоит задача многоклассовой классификации. Вы будете применять ее в следующих лаборатоных.

$ softmax(z)_i = \frac{e^{z_i}}{\sum_{k=1}^K e^{z_k}} $
- $ \text{для } x \in \mathbb{R}^{1\times n} \text{,     } softmax(x) = softmax(\begin{bmatrix}
    x_1  &&
    x_2 &&
    ...  &&
    x_n  
\end{bmatrix}) = \begin{bmatrix}
     \frac{e^{x_1}}{\sum_{j}e^{x_j}}  &&
    \frac{e^{x_2}}{\sum_{j}e^{x_j}}  &&
    ...  &&
    \frac{e^{x_n}}{\sum_{j}e^{x_j}}
\end{bmatrix} $

- $\text{для массива } x \in \mathbb{R}^{m \times n} \text{, где  $x_{ij}$ - это элемент i-й строки и j-го столбца матрицы $x$: }$  $$softmax(x) = softmax\begin{bmatrix}
    x_{11} & x_{12} & x_{13} & \dots  & x_{1n} \\
    x_{21} & x_{22} & x_{23} & \dots  & x_{2n} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    x_{m1} & x_{m2} & x_{m3} & \dots  & x_{mn}
\end{bmatrix} = \begin{bmatrix}
    \frac{e^{x_{11}}}{\sum_{j}e^{x_{1j}}} & \frac{e^{x_{12}}}{\sum_{j}e^{x_{1j}}} & \frac{e^{x_{13}}}{\sum_{j}e^{x_{1j}}} & \dots  & \frac{e^{x_{1n}}}{\sum_{j}e^{x_{1j}}} \\
    \frac{e^{x_{21}}}{\sum_{j}e^{x_{2j}}} & \frac{e^{x_{22}}}{\sum_{j}e^{x_{2j}}} & \frac{e^{x_{23}}}{\sum_{j}e^{x_{2j}}} & \dots  & \frac{e^{x_{2n}}}{\sum_{j}e^{x_{2j}}} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    \frac{e^{x_{m1}}}{\sum_{j}e^{x_{mj}}} & \frac{e^{x_{m2}}}{\sum_{j}e^{x_{mj}}} & \frac{e^{x_{m3}}}{\sum_{j}e^{x_{mj}}} & \dots  & \frac{e^{x_{mn}}}{\sum_{j}e^{x_{mj}}}
\end{bmatrix} = \begin{pmatrix}
    softmax\text{(first row of x)}  \\
    softmax\text{(second row of x)} \\
    ...  \\
    softmax\text{(last row of x)} \\
\end{pmatrix} $$

In [ ]:
def softmax(x):
    """Вычисление функции softmax для каждой строки массива x (m,n).
    """

    # YOUR CODE HERE
    raise NotImplementedError()

    return s

In [ ]:
x = np.array([
    [9, 2, 5, 0, 0],
    [7, 5, 0, 0 ,0]])
res = softmax(x)
print("softmax(x) = " + str(res))

assert round(res[1,0], 2) == 0.88
assert round(res[0,3], 5) == 0.00012
assert round(res[0,0], 2) == 0.98

### 1.7 - Нормализация строк

Было бы проблематично передать в нейронную сеть значения, имеющие самые разные диапазоны. Сеть, конечно, сможет автоматически адаптироваться к таким разнородным данным, однако это усложнит обучение. На практике к таким данным принято применять нормализацию.

Целью нормализации значений является преобразование данных к виду, который наиболее подходит для обработки, т.е. данные, поступающие на вход, должны иметь числовой тип, а их значения должны быть распределены в определенном диапазоне.

На практике наиболее распространены следующие методы нормализации признаков:
- минимакс, линейное преобразование данных в диапазоне \[0..1\], где минимальное и максимальное масштабируемые значения соответствуют 0 и 1 соответственно: $$x_{norm}=\frac{x-x_{min}}{x_{max}+x_{min}}\tag{3}$$
- Z-масштабирование, для каждого признака во входных данных из каждого значения вычитается среднее по этому признаку $\mu$, и разность делится на стандартное отклонение $\sigma$, в результате признак центрируется по нулевому значению и имеет стандартное отклонение, равное единице: $$x_{norm}=\frac{x-\mu}{\sigma}\tag{4}$$
- масштабирование векторов данных на основе нормы вектора $$ x_{norm}=\frac{x}{\| x\|}\tag{5}$$

Рассмотрим нормализацию путем деления векторов (строчек) данных на норму.
Например, если $$x =
\begin{bmatrix}
    0 & 3 & 4 \\
    2 & 6 & 4 \\
\end{bmatrix}\tag{6}$$ тогда норма$$\| x\| = np.linalg.norm(x, axis = 1, keepdims = True) = \begin{bmatrix}
    5 \\
    \sqrt{56} \\
\end{bmatrix}\tag{7} $$и        $$ x\_normalized = \frac{x}{\| x\|} = \begin{bmatrix}
    0 & \frac{3}{5} & \frac{4}{5} \\
    \frac{2}{\sqrt{56}} & \frac{6}{\sqrt{56}} & \frac{4}{\sqrt{56}} \\
\end{bmatrix}\tag{8}$$

**Задание**: Реализуте функцию normalize() для масштабирования строк матрицы x.

In [ ]:
def normalize(x):
    # YOUR CODE HERE
    raise NotImplementedError()

    return x

In [ ]:
x = np.array([
    [0, 3, 4],
    [1, 6, 4]])
res = normalize(x)
print("normalize(x) = " + str(res))
assert res[0,1] == 0.6
assert round(res[1,1], 2) == 0.82
assert res[0,2] == 0.8

## 2. Векторизация

В области глубокого обучения часто приходится работать с огромными массивами данных. Поэтому неоптимальная в вычислительном отношении функция может стать узким местом в вашем алгоритме и привести к существенному увеличению времени работы модели. Для написания эффективного в вычислительном отношении кода мы будем использовать *векторизацию*.

**Задание:** Постарайтесь определить разницу между следующими функциями умножения матриц. Реализуйте по аналогии эти функции с помощью векторизированных функций Numpy. Сделайте вывод.

In [ ]:
import time

x1 = [4, 2, 5, 7, 0, 8, 5, 0, 7, 0, 9, 2, 5, 1, 0]
x2 = [9, 2, 3, 9, 0, 9, 2, 5, 0, 0, 3, 2, 5, 0, 1]

### Cкалярное произведение векторов ###
tic = time.process_time()
dot = 0
for i in range(len(x1)):
    dot+= x1[i]*x2[i]
toc = time.process_time()
print ("dot = " + str(dot) + "\n ----- Время вычисления = " + str(1000*(toc - tic)) + "мс")

### Внешнее произведение двух векторов ###
tic = time.process_time()
outer = np.zeros((len(x1),len(x2)))
for i in range(len(x1)):
    for j in range(len(x2)):
        outer[i,j] = x1[i]*x2[j]
toc = time.process_time()
print ("outer = " + str(outer) + "\n ----- Время вычисления = " + str(1000*(toc - tic)) + "мс")

### Поэлементное уможение векторов ###
tic = time.process_time()
mul = np.zeros(len(x1))
for i in range(len(x1)):
    mul[i] = x1[i]*x2[i]
toc = time.process_time()
print ("elementwise multiplication = " + str(mul) + "\n ----- Время вычисления = " + str(1000*(toc - tic)) + "мс")

### Матричное умножение ###
W = np.random.rand(6,len(x1))
tic = time.process_time()
gdot = np.zeros(W.shape[0])
for i in range(W.shape[0]):
    for j in range(len(x1)):
        gdot[i] += W[i,j]*x1[j]
toc = time.process_time()
print ("gdot = " + str(gdot) + "\n ----- Время вычисления = " + str(1000*(toc - tic)) + "мс")

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Задание**: для закрепления полученных знаний о библиотеке numpy реализуйте с ее помощью выражения из файла "Варианты.pdf" согласно вашему варианту.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Контрольные вопросы:
1. Описать работу формального нейрона?
2. Что такое тензор. Какие бывают тензоры?
3. Что такое функция активации и какова ее задача?
4. Какие бывают функции активации? В чем их отличие друг от друга?
5. Для чего необходимо рассчитывать производную функции активации?
6. Для чего нужно укладывание? Сформулируйте правило укладывания.
7. Что представляет собой функцияи softmax?
8. Зачем нужна процедура нормализации? Какие существуют методы нормализации данных?
9. Зачем нужна процедура векторизации?

<font color='blue'>
Источники литературы для успешного выполнения лабораторной работы:
<br>-  Шолле Ф.Глубокое обучение на Python / Ф. Шолле – СПб.: Питер, 2018.–
400c.
<br>- Маккини У. Python И анализ данных / пер. с анг. А.А. Слинкина. - М.: ДМК Пресс, 2020. - 540 с.: ил. [Лань: доступ через ЭБС НТБ]
<br>- Спицын В.Г. Интеллектуальные системы: учебное пособие / В.Г. Спицын, Ю.Р. Цой; Томский политехнический университет. – Томск: Изд-во Томского политехнического университета, 2012. – 176 с.